In [1]:
# Install any necessary packages (like wordcloud for an optional bonus task)
!pip install -q pandas matplotlib seaborn plotly wordcloud

# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Set visual style for charts
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


In [ ]:
from google.colab import files
uploaded = files.upload()

# Load the dataset into a DataFrame
import io
file_name = list(uploaded.keys())[0] # Gets the name of your uploaded file
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

# Show basic info
print("Dataset loaded successfully!")
print(f"Shape: {df.shape}") # Shows (number of rows, number of columns)
print("\nFirst look at the data:")
print(df.head())

In [ ]:
# Initial Data Check
print("=== STEP 1: INITIAL DATA CHECK ===")
print(f"Dataset Shape: {df.shape} rows, {df.shape[1]} columns")
print("\n--- Column Data Types ---")
print(df.dtypes)
print("\n--- First Look at Missing Values ---")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])  # Show only columns with missing data
print("\n--- Sample of Problematic 'duration' Column ---")
print(df[['type', 'duration']].head(10))

In [ ]:
# Handle Missing Data
print("=== STEP 2: HANDLING MISSING DATA ===")

# 1. Drop less important columns if they have too many missing values (like 'director')
# We'll fill others with placeholders.
df_clean = df.copy()

# Fill missing text fields with 'Unknown'
text_columns_to_fill = ['director', 'cast', 'country', 'rating']
for col in text_columns_to_fill:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna('Unknown')
        missing_before = df[col].isnull().sum()
        missing_after = df_clean[col].isnull().sum()
        print(f"Filled {missing_before} missing values in '{col}'.")

# For 'date_added', we might want to drop rows if analysis is time-based,
# or fill with a placeholder. Let's just drop for simplicity in a date analysis.
rows_before = len(df_clean)
df_clean = df_clean.dropna(subset=['date_added'])
rows_after = len(df_clean)
print(f"\nDropped {rows_before - rows_after} rows with missing 'date_added'.")

print(f"\n--- Missing Values After Cleaning ---")
print(df_clean.isnull().sum().sum(), "total missing values remaining.")

In [ ]:
# Fix Data Types and Formatting
print("=== STEP 3: FIXING DATA TYPES AND FORMATTING ===")

# 1. Clean 'date_added' column: remove leading/trailing whitespace first
df_clean['date_added'] = df_clean['date_added'].str.strip()
print("✅ Stripped whitespace from 'date_added'.")

# 2. Now convert to datetime with flexible parsing
df_clean['date_added'] = pd.to_datetime(df_clean['date_added'], errors='coerce', format='mixed')
print(f"✅ Converted 'date_added' to datetime. {df_clean['date_added'].isna().sum()} dates failed to parse.")

# 3. Extract year and month from successfully parsed dates
df_clean['year_added'] = df_clean['date_added'].dt.year
df_clean['month_added'] = df_clean['date_added'].dt.month_name()
print("✅ Extracted 'year_added' and 'month_added'.")

# 4. Fix the 'duration' column issue
print("\n--- Handling 'duration' Column ---")
def split_duration(row):
    try:
        if pd.isna(row['duration']):
            return pd.Series([None, None])
        if row['type'] == 'Movie':
            # Extract numbers from strings like "90 min"
            num_part = str(row['duration']).split()[0]
            return pd.Series([int(num_part) if num_part.isdigit() else None, None])
        else:  # TV Show
            # Extract numbers from strings like "2 Seasons"
            num_part = str(row['duration']).split()[0]
            return pd.Series([None, int(num_part) if num_part.isdigit() else None])
    except Exception as e:
        # If anything goes wrong, return None values
        return pd.Series([None, None])

# Apply the function
df_clean[['movie_duration_min', 'show_num_seasons']] = df_clean.apply(split_duration, axis=1)
print(f"✅ Split 'duration' column:")
print(f"   - {df_clean['movie_duration_min'].notna().sum()} movies have duration in minutes")
print(f"   - {df_clean['show_num_seasons'].notna().sum()} TV shows have season count")

# 5. Handle the 'listed_in' (genres) column
df_clean['genres_list'] = df_clean['listed_in'].str.split(', ')

print("\n--- New/Updated Columns Preview ---")
print(df_clean[['type', 'date_added', 'duration', 'movie_duration_min', 'show_num_seasons', 'genres_list']].head(10))

In [ ]:
# Clean Complex Columns
print("=== STEP 4: CLEANING COMPLEX COLUMNS ===")

# 1. The 'country' column often has multiple entries like "United States, India, United Kingdom"
# We'll keep only the first (primary) country for simpler analysis.
df_clean['primary_country'] = df_clean['country'].str.split(',').str[0].str.strip()
print("✅ Extracted 'primary_country' (first country listed).")

# 2. Standardize text in 'rating' by stripping whitespace (some might have extra spaces)
df_clean['rating'] = df_clean['rating'].str.strip()
print("✅ Stripped whitespace from 'rating' column.")

# 3. Clean 'cast': similar to country, we'll just take the first listed actor for a "main star" analysis.
df_clean['main_cast'] = df_clean['cast'].str.split(',').str[0].str.strip()
print("✅ Extracted 'main_cast' (first actor listed).")

print(f"\n--- Cleaned Data Shape ---")
print(df_clean.shape)

In [ ]:
# Final Verification
print("=== STEP 5: FINAL VERIFICATION ===")
print("\n--- Cleaned Dataset Info ---")
print(df_clean.info())

print("\n--- Sample of Cleaned Data ---")
print(df_clean[['type', 'primary_country', 'rating', 'movie_duration_min', 'show_num_seasons']].head(10))

print("\n--- Summary Statistics for New Numeric Columns ---")
print(df_clean[['movie_duration_min', 'show_num_seasons']].describe())

# Save the cleaned dataframe to a new CSV if you want to use it later
df_clean.to_csv('netflix_titles_cleaned.csv', index=False)
print("\n✅ Data cleaning complete! Cleaned dataset saved as 'netflix_titles_cleaned.csv'.")
print("🎯 You are now ready for efficient analysis!")

In [ ]:
print("--- QUESTION 1: Data Overview ---")
print(f"Total titles in the dataset: {len(df)}")

# Count of Movies vs. TV Shows
type_counts = df['type'].value_counts()
print(f"\nNumber of Movies: {type_counts.get('Movie', 0)}")
print(f"Number of TV Shows: {type_counts.get('TV Show', 0)}")

# Simple visualization
type_counts.plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Movies vs. TV Shows on Netflix')
plt.xlabel('Type')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

In [ ]:
print("--- QUESTION 2: Content Trends Over Time ---")

# Convert 'date_added' to datetime and extract the year
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
df['year_added'] = df['date_added'].dt.year

# Count titles added per year
additions_per_year = df['year_added'].value_counts().sort_index()

# Plot the trend
plt.plot(additions_per_year.index, additions_per_year.values, marker='o')
plt.title('Netflix Content Added Per Year')
plt.xlabel('Year')
plt.ylabel('Number of Titles Added')
plt.grid(True)
plt.show()

# Optional: Separate trend for Movies and TV Shows
plt.figure(figsize=(12, 5))
for content_type in ['Movie', 'TV Show']:
    subset = df[df['type'] == content_type]
    trend = subset['year_added'].value_counts().sort_index()
    plt.plot(trend.index, trend.values, marker='o', label=content_type)

plt.title('Content Added Per Year (by Type)')
plt.xlabel('Year')
plt.ylabel('Count')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print("--- QUESTION 3: Which countries produce the most Netflix content? ---\n")

# 1. Check and clean the 'country' column (some entries have multiple countries separated by commas)
# We'll take the first listed country as the primary one.
df['primary_country'] = df['country'].str.split(',').str[0].str.strip()

# 2. Count titles by primary country, get the top 10
top_countries = df['primary_country'].value_counts().head(10)

print("Top 10 Content-Producing Countries:")
print(top_countries.to_string())

# 3. Create a bar chart
plt.figure(figsize=(12, 6))
bars = plt.barh(top_countries.index, top_countries.values, color='lightcoral')
plt.gca().invert_yaxis() # Display the highest count at the top
plt.title('Top 10 Countries by Number of Netflix Titles')
plt.xlabel('Number of Titles')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Add the count numbers to the end of each bar
for bar in bars:
    width = bar.get_width()
    plt.text(width + 5, bar.get_y() + bar.get_height()/2,
             f'{int(width)}', va='center')

plt.tight_layout()
plt.show()

# 4. Bonus: Check for missing values
missing_country_pct = (df['country'].isna().sum() / len(df)) * 100
print(f"\nNote: {df['country'].isna().sum():,} entries ({missing_country_pct:.1f}%) have no country data.")

In [ ]:
print("--- QUESTION 4: What are the most common content ratings? ---\n")

# 1. Count the ratings
rating_counts = df['rating'].value_counts()

print("All Content Ratings (Sorted by Frequency):")
print(rating_counts.to_string())

# 2. Plot the top 10 ratings
top_ratings = rating_counts.head(10)

plt.figure(figsize=(10, 6))
bars = plt.bar(top_ratings.index, top_ratings.values, color='mediumseagreen')
plt.title('Top 10 Most Common Content Ratings on Netflix')
plt.xlabel('Rating')
plt.ylabel('Number of Titles')
plt.xticks(rotation=45, ha='right')

# Add count labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 5,
             f'{int(height)}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# 3. Compare ratings for Movies vs. TV Shows
print("\n--- Rating Distribution by Type (Top 5 Ratings for Each) ---")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for i, content_type in enumerate(['Movie', 'TV Show']):
    subset = df[df['type'] == content_type]
    type_ratings = subset['rating'].value_counts().head(5)

    axes[i].bar(type_ratings.index, type_ratings.values, color='steelblue' if i==0 else 'darkorange')
    axes[i].set_title(f'Top 5 Ratings: {content_type}s')
    axes[i].set_xlabel('Rating')
    axes[i].set_ylabel('Count')
    axes[i].tick_params(axis='x', rotation=45)

    # Add labels
    for bar in axes[i].patches:
        axes[i].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 5,
                    f'{int(bar.get_height())}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
files.download('netflix_titles_cleaned.csv')